In [1]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 31 kB/s 
     |████████████████████████████████| 198 kB 69.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=246098b7a2590da92280c02e014f2767abe659d71a694d420d1363c40c4587f9
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [2]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import date, timedelta, datetime
import time

##**INITIALIZING SPARKSESSION**

In [3]:
#Initialization of a Spark Session
sc = SparkSession.builder.appName("Pyspark-sql").getOrCreate()

##**CREATING DATAFRAMES**

In [6]:
dataframe = sc.read.csv("data/student-mat.csv",header=True)

In [7]:
dataframe.show()

+------+---+---+-------+-------+-------+----+----+--------+--------+----------+--------+----------+---------+--------+---------+------+----+----------+-------+------+--------+--------+------+--------+-----+----+----+------+--------+---+---+---+
|school|sex|age|address|famsize|Pstatus|Medu|Fedu|    Mjob|    Fjob|    reason|guardian|traveltime|studytime|failures|schoolsup|famsup|paid|activities|nursery|higher|internet|romantic|famrel|freetime|goout|Dalc|Walc|health|absences| G1| G2| G3|
+------+---+---+-------+-------+-------+----+----+--------+--------+----------+--------+----------+---------+--------+---------+------+----+----------+-------+------+--------+--------+------+--------+-----+----+----+------+--------+---+---+---+
|    GP|  F| 18|      U|    GT3|      A|   4|   4| at_home| teacher|    course|  mother|         2|        2|       0|      yes|    no|  no|        no|    yes|   yes|      no|      no|     4|       3|    4|   1|   1|     3|       6|  5|  6|  6|
|    GP|  F| 17|    

##**DUPLICATE VALUES**

In [8]:
dataframe_dropdup = dataframe.dropDuplicates()

In [9]:
dataframe_dropdup.show(10)

+------+---+---+-------+-------+-------+----+----+-------+--------+----------+--------+----------+---------+--------+---------+------+----+----------+-------+------+--------+--------+------+--------+-----+----+----+------+--------+---+---+---+
|school|sex|age|address|famsize|Pstatus|Medu|Fedu|   Mjob|    Fjob|    reason|guardian|traveltime|studytime|failures|schoolsup|famsup|paid|activities|nursery|higher|internet|romantic|famrel|freetime|goout|Dalc|Walc|health|absences| G1| G2| G3|
+------+---+---+-------+-------+-------+----+----+-------+--------+----------+--------+----------+---------+--------+---------+------+----+----------+-------+------+--------+--------+------+--------+-----+----+----+------+--------+---+---+---+
|    GP|  F| 17|      R|    GT3|      T|   3|   4|at_home|services|    course|  father|         1|        3|       0|       no|   yes| yes|       yes|     no|   yes|     yes|      no|     4|       3|    4|   2|   5|     5|       0| 11| 11| 10|
|    GP|  F| 15|      U|

In [10]:
dataframe.columns

['school',
 'sex',
 'age',
 'address',
 'famsize',
 'Pstatus',
 'Medu',
 'Fedu',
 'Mjob',
 'Fjob',
 'reason',
 'guardian',
 'traveltime',
 'studytime',
 'failures',
 'schoolsup',
 'famsup',
 'paid',
 'activities',
 'nursery',
 'higher',
 'internet',
 'romantic',
 'famrel',
 'freetime',
 'goout',
 'Dalc',
 'Walc',
 'health',
 'absences',
 'G1',
 'G2',
 'G3']

##**QUERIES**

**Select**

In [15]:
# Select Operation

dataframe.select("school").show(10) #Show all entries in sex column

dataframe.select("school", "sex", "famsize", "Pstatus").show(10)  #Show all entries in school, sex, famsize, Pstatus columns

+------+
|school|
+------+
|    GP|
|    GP|
|    GP|
|    GP|
|    GP|
|    GP|
|    GP|
|    GP|
|    GP|
|    GP|
+------+
only showing top 10 rows

+------+---+-------+-------+
|school|sex|famsize|Pstatus|
+------+---+-------+-------+
|    GP|  F|    GT3|      A|
|    GP|  F|    GT3|      T|
|    GP|  F|    LE3|      T|
|    GP|  F|    GT3|      T|
|    GP|  F|    GT3|      T|
|    GP|  M|    LE3|      T|
|    GP|  M|    LE3|      T|
|    GP|  F|    GT3|      A|
|    GP|  M|    LE3|      A|
|    GP|  M|    GT3|      T|
+------+---+-------+-------+
only showing top 10 rows



When

In [16]:
# Show sex and assign 0 or 1 depending on Male or Female

dataframe.select("sex", when(dataframe.sex != 'F', 1).otherwise(0)).show(10)

+---+-------------------------------------------+
|sex|CASE WHEN (NOT (sex = F)) THEN 1 ELSE 0 END|
+---+-------------------------------------------+
|  F|                                          0|
|  F|                                          0|
|  F|                                          0|
|  F|                                          0|
|  F|                                          0|
|  M|                                          1|
|  M|                                          1|
|  F|                                          0|
|  M|                                          1|
|  M|                                          1|
+---+-------------------------------------------+
only showing top 10 rows



In [18]:
# Show rows of students whose father's education is bwteen 3 – secondary education or 4 – higher education)

dataframe [dataframe.Fedu.isin("3","4")].show(5)

+------+---+---+-------+-------+-------+----+----+--------+-------+----------+--------+----------+---------+--------+---------+------+----+----------+-------+------+--------+--------+------+--------+-----+----+----+------+--------+---+---+---+
|school|sex|age|address|famsize|Pstatus|Medu|Fedu|    Mjob|   Fjob|    reason|guardian|traveltime|studytime|failures|schoolsup|famsup|paid|activities|nursery|higher|internet|romantic|famrel|freetime|goout|Dalc|Walc|health|absences| G1| G2| G3|
+------+---+---+-------+-------+-------+----+----+--------+-------+----------+--------+----------+---------+--------+---------+------+----+----------+-------+------+--------+--------+------+--------+-----+----+----+------+--------+---+---+---+
|    GP|  F| 18|      U|    GT3|      A|   4|   4| at_home|teacher|    course|  mother|         2|        2|       0|      yes|    no|  no|        no|    yes|   yes|      no|      no|     4|       3|    4|   1|   1|     3|       6|  5|  6|  6|
|    GP|  F| 16|      U|

**Like**

In [22]:
# # Show school and Mother's job is TRUE if Mjob has "er" word in Mjob

dataframe.select("Mjob", "school", dataframe.Mjob.like("%er")).show(15)

+--------+------+-------------+
|    Mjob|school|Mjob LIKE %er|
+--------+------+-------------+
| at_home|    GP|        false|
| at_home|    GP|        false|
| at_home|    GP|        false|
|  health|    GP|        false|
|   other|    GP|         true|
|services|    GP|        false|
|   other|    GP|         true|
|   other|    GP|         true|
|services|    GP|        false|
|   other|    GP|         true|
| teacher|    GP|         true|
|services|    GP|        false|
|  health|    GP|        false|
| teacher|    GP|         true|
|   other|    GP|         true|
+--------+------+-------------+
only showing top 15 rows



**StartsWith - EndsWith**

In [25]:
dataframe.select("Mjob", "school", dataframe.Mjob.startswith("at_")).show(5)

dataframe.select("Fjob", "school", dataframe.Fjob.endswith("er")).show(5)

+-------+------+---------------------+
|   Mjob|school|startswith(Mjob, at_)|
+-------+------+---------------------+
|at_home|    GP|                 true|
|at_home|    GP|                 true|
|at_home|    GP|                 true|
| health|    GP|                false|
|  other|    GP|                false|
+-------+------+---------------------+
only showing top 5 rows

+--------+------+------------------+
|    Fjob|school|endswith(Fjob, er)|
+--------+------+------------------+
| teacher|    GP|              true|
|   other|    GP|              true|
|   other|    GP|              true|
|services|    GP|             false|
|   other|    GP|              true|
+--------+------+------------------+
only showing top 5 rows



**Substring**

In [26]:
dataframe.select(dataframe.Mjob.substr(1, 3).alias("school")).show()

+------+
|school|
+------+
|   at_|
|   at_|
|   at_|
|   hea|
|   oth|
|   ser|
|   oth|
|   oth|
|   ser|
|   oth|
|   tea|
|   ser|
|   hea|
|   tea|
|   oth|
|   hea|
|   ser|
|   oth|
|   ser|
|   hea|
+------+
only showing top 20 rows



**INSPECT DATA**

In [27]:
# Returns dataframe column names and data types
dataframe.dtypes

[('school', 'string'),
 ('sex', 'string'),
 ('age', 'string'),
 ('address', 'string'),
 ('famsize', 'string'),
 ('Pstatus', 'string'),
 ('Medu', 'string'),
 ('Fedu', 'string'),
 ('Mjob', 'string'),
 ('Fjob', 'string'),
 ('reason', 'string'),
 ('guardian', 'string'),
 ('traveltime', 'string'),
 ('studytime', 'string'),
 ('failures', 'string'),
 ('schoolsup', 'string'),
 ('famsup', 'string'),
 ('paid', 'string'),
 ('activities', 'string'),
 ('nursery', 'string'),
 ('higher', 'string'),
 ('internet', 'string'),
 ('romantic', 'string'),
 ('famrel', 'string'),
 ('freetime', 'string'),
 ('goout', 'string'),
 ('Dalc', 'string'),
 ('Walc', 'string'),
 ('health', 'string'),
 ('absences', 'string'),
 ('G1', 'string'),
 ('G2', 'string'),
 ('G3', 'string')]

In [28]:
# Displays the content of dataframe
dataframe.show()

+------+---+---+-------+-------+-------+----+----+--------+--------+----------+--------+----------+---------+--------+---------+------+----+----------+-------+------+--------+--------+------+--------+-----+----+----+------+--------+---+---+---+
|school|sex|age|address|famsize|Pstatus|Medu|Fedu|    Mjob|    Fjob|    reason|guardian|traveltime|studytime|failures|schoolsup|famsup|paid|activities|nursery|higher|internet|romantic|famrel|freetime|goout|Dalc|Walc|health|absences| G1| G2| G3|
+------+---+---+-------+-------+-------+----+----+--------+--------+----------+--------+----------+---------+--------+---------+------+----+----------+-------+------+--------+--------+------+--------+-----+----+----+------+--------+---+---+---+
|    GP|  F| 18|      U|    GT3|      A|   4|   4| at_home| teacher|    course|  mother|         2|        2|       0|      yes|    no|  no|        no|    yes|   yes|      no|      no|     4|       3|    4|   1|   1|     3|       6|  5|  6|  6|
|    GP|  F| 17|    

In [29]:
# Return first n rows
dataframe.head()

Row(school='GP', sex='F', age='18', address='U', famsize='GT3', Pstatus='A', Medu='4', Fedu='4', Mjob='at_home', Fjob='teacher', reason='course', guardian='mother', traveltime='2', studytime='2', failures='0', schoolsup='yes', famsup='no', paid='no', activities='no', nursery='yes', higher='yes', internet='no', romantic='no', famrel='4', freetime='3', goout='4', Dalc='1', Walc='1', health='3', absences='6', G1='5', G2='6', G3='6')

In [30]:
# Returns first row
dataframe.first()

Row(school='GP', sex='F', age='18', address='U', famsize='GT3', Pstatus='A', Medu='4', Fedu='4', Mjob='at_home', Fjob='teacher', reason='course', guardian='mother', traveltime='2', studytime='2', failures='0', schoolsup='yes', famsup='no', paid='no', activities='no', nursery='yes', higher='yes', internet='no', romantic='no', famrel='4', freetime='3', goout='4', Dalc='1', Walc='1', health='3', absences='6', G1='5', G2='6', G3='6')

In [31]:
# Return first n rows
dataframe.take(5)

[Row(school='GP', sex='F', age='18', address='U', famsize='GT3', Pstatus='A', Medu='4', Fedu='4', Mjob='at_home', Fjob='teacher', reason='course', guardian='mother', traveltime='2', studytime='2', failures='0', schoolsup='yes', famsup='no', paid='no', activities='no', nursery='yes', higher='yes', internet='no', romantic='no', famrel='4', freetime='3', goout='4', Dalc='1', Walc='1', health='3', absences='6', G1='5', G2='6', G3='6'),
 Row(school='GP', sex='F', age='17', address='U', famsize='GT3', Pstatus='T', Medu='1', Fedu='1', Mjob='at_home', Fjob='other', reason='course', guardian='father', traveltime='1', studytime='2', failures='0', schoolsup='no', famsup='yes', paid='no', activities='no', nursery='no', higher='yes', internet='yes', romantic='no', famrel='5', freetime='3', goout='3', Dalc='1', Walc='1', health='3', absences='4', G1='5', G2='5', G3='6'),
 Row(school='GP', sex='F', age='15', address='U', famsize='LE3', Pstatus='T', Medu='1', Fedu='1', Mjob='at_home', Fjob='other', re

In [32]:
# Computes summary statistics
dataframe.describe().show()

+-------+------+----+------------------+-------+-------+-------+------------------+------------------+-------+-------+----------+--------+------------------+------------------+------------------+---------+------+----+----------+-------+------+--------+--------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+------------------+
|summary|school| sex|               age|address|famsize|Pstatus|              Medu|              Fedu|   Mjob|   Fjob|    reason|guardian|        traveltime|         studytime|          failures|schoolsup|famsup|paid|activities|nursery|higher|internet|romantic|            famrel|          freetime|             goout|              Dalc|              Walc|            health|         absences|                G1|                G2|                G3|
+-------+------+----+------------------+-------+-------+-------+------------------+---------------

In [33]:
# Returns columns of dataframe
dataframe.columns

['school',
 'sex',
 'age',
 'address',
 'famsize',
 'Pstatus',
 'Medu',
 'Fedu',
 'Mjob',
 'Fjob',
 'reason',
 'guardian',
 'traveltime',
 'studytime',
 'failures',
 'schoolsup',
 'famsup',
 'paid',
 'activities',
 'nursery',
 'higher',
 'internet',
 'romantic',
 'famrel',
 'freetime',
 'goout',
 'Dalc',
 'Walc',
 'health',
 'absences',
 'G1',
 'G2',
 'G3']

In [34]:
# Counts the number of rows in dataframe
dataframe.count()

395

In [35]:
# Counts the number of distinct rows in dataframe
dataframe.distinct().count()

395

##**ADD, UPDATE & REMOVE COLUMNS**

In [37]:
# Lit() is required while we are creating columns with exact values.

dataframe = dataframe.withColumn('new_column', lit('This is a new column'))
dataframe.show(5)

+------+---+---+-------+-------+-------+----+----+-------+--------+------+--------+----------+---------+--------+---------+------+----+----------+-------+------+--------+--------+------+--------+-----+----+----+------+--------+---+---+---+--------------------+
|school|sex|age|address|famsize|Pstatus|Medu|Fedu|   Mjob|    Fjob|reason|guardian|traveltime|studytime|failures|schoolsup|famsup|paid|activities|nursery|higher|internet|romantic|famrel|freetime|goout|Dalc|Walc|health|absences| G1| G2| G3|          new_column|
+------+---+---+-------+-------+-------+----+----+-------+--------+------+--------+----------+---------+--------+---------+------+----+----------+-------+------+--------+--------+------+--------+-----+----+----+------+--------+---+---+---+--------------------+
|    GP|  F| 18|      U|    GT3|      A|   4|   4|at_home| teacher|course|  mother|         2|        2|       0|      yes|    no|  no|        no|    yes|   yes|      no|      no|     4|       3|    4|   1|   1|     3

In [40]:
# Update column 'Medu' with 'Mothers_Edu'

dataframe = dataframe.withColumnRenamed('Medu', 'Mother_Edu')
dataframe.show(5)

+------+---+---+-------+-------+-------+----------+----+-------+--------+------+--------+----------+---------+--------+---------+------+----+----------+-------+------+--------+--------+------+--------+-----+----+----+------+--------+---+---+---+--------------------+
|school|sex|age|address|famsize|Pstatus|Mother_Edu|Fedu|   Mjob|    Fjob|reason|guardian|traveltime|studytime|failures|schoolsup|famsup|paid|activities|nursery|higher|internet|romantic|famrel|freetime|goout|Dalc|Walc|health|absences| G1| G2| G3|          new_column|
+------+---+---+-------+-------+-------+----------+----+-------+--------+------+--------+----------+---------+--------+---------+------+----+----------+-------+------+--------+--------+------+--------+-----+----+----+------+--------+---+---+---+--------------------+
|    GP|  F| 18|      U|    GT3|      A|         4|   4|at_home| teacher|course|  mother|         2|        2|       0|      yes|    no|  no|        no|    yes|   yes|      no|      no|     4|       

In [39]:
dataframe_remove = dataframe.drop("address", "famsize").show(5)

dataframe_remove2 = dataframe.drop(dataframe.address).drop(dataframe.famsize).show(5)

+------+---+---+-------+----------+----+-------+--------+------+--------+----------+---------+--------+---------+------+----+----------+-------+------+--------+--------+------+--------+-----+----+----+------+--------+---+---+---+--------------------+
|school|sex|age|Pstatus|Mother_Edu|Fedu|   Mjob|    Fjob|reason|guardian|traveltime|studytime|failures|schoolsup|famsup|paid|activities|nursery|higher|internet|romantic|famrel|freetime|goout|Dalc|Walc|health|absences| G1| G2| G3|          new_column|
+------+---+---+-------+----------+----+-------+--------+------+--------+----------+---------+--------+---------+------+----+----------+-------+------+--------+--------+------+--------+-----+----+----+------+--------+---+---+---+--------------------+
|    GP|  F| 18|      A|         4|   4|at_home| teacher|course|  mother|         2|        2|       0|      yes|    no|  no|        no|    yes|   yes|      no|      no|     4|       3|    4|   1|   1|     3|       6|  5|  6|  6|This is a new colu

##**GROUPBY**

In [41]:
# Group by sex, count the students of the different genders in the groups
dataframe.groupBy("sex").count().show(10)

+---+-----+
|sex|count|
+---+-----+
|  F|  208|
|  M|  187|
+---+-----+



##**MISSIING** AND REPLACING VALUES

In [42]:
# Replace null values
dataframe.na.fill(50).show(5)

+------+---+---+-------+-------+-------+----------+----+-------+--------+------+--------+----------+---------+--------+---------+------+----+----------+-------+------+--------+--------+------+--------+-----+----+----+------+--------+---+---+---+--------------------+
|school|sex|age|address|famsize|Pstatus|Mother_Edu|Fedu|   Mjob|    Fjob|reason|guardian|traveltime|studytime|failures|schoolsup|famsup|paid|activities|nursery|higher|internet|romantic|famrel|freetime|goout|Dalc|Walc|health|absences| G1| G2| G3|          new_column|
+------+---+---+-------+-------+-------+----------+----+-------+--------+------+--------+----------+---------+--------+---------+------+----+----------+-------+------+--------+--------+------+--------+-----+----+----+------+--------+---+---+---+--------------------+
|    GP|  F| 18|      U|    GT3|      A|         4|   4|at_home| teacher|course|  mother|         2|        2|       0|      yes|    no|  no|        no|    yes|   yes|      no|      no|     4|       

In [43]:
# Return new dataframe restricting rows with null values
dataframe.na.drop().show(5)

+------+---+---+-------+-------+-------+----------+----+-------+--------+------+--------+----------+---------+--------+---------+------+----+----------+-------+------+--------+--------+------+--------+-----+----+----+------+--------+---+---+---+--------------------+
|school|sex|age|address|famsize|Pstatus|Mother_Edu|Fedu|   Mjob|    Fjob|reason|guardian|traveltime|studytime|failures|schoolsup|famsup|paid|activities|nursery|higher|internet|romantic|famrel|freetime|goout|Dalc|Walc|health|absences| G1| G2| G3|          new_column|
+------+---+---+-------+-------+-------+----------+----+-------+--------+------+--------+----------+---------+--------+---------+------+----+----------+-------+------+--------+--------+------+--------+-----+----+----+------+--------+---+---+---+--------------------+
|    GP|  F| 18|      U|    GT3|      A|         4|   4|at_home| teacher|course|  mother|         2|        2|       0|      yes|    no|  no|        no|    yes|   yes|      no|      no|     4|       

In [44]:
# Return new dataframe replacing one value with another
dataframe.na.replace(10, 20).show(5)

+------+---+---+-------+-------+-------+----------+----+-------+--------+------+--------+----------+---------+--------+---------+------+----+----------+-------+------+--------+--------+------+--------+-----+----+----+------+--------+---+---+---+--------------------+
|school|sex|age|address|famsize|Pstatus|Mother_Edu|Fedu|   Mjob|    Fjob|reason|guardian|traveltime|studytime|failures|schoolsup|famsup|paid|activities|nursery|higher|internet|romantic|famrel|freetime|goout|Dalc|Walc|health|absences| G1| G2| G3|          new_column|
+------+---+---+-------+-------+-------+----------+----+-------+--------+------+--------+----------+---------+--------+---------+------+----+----------+-------+------+--------+--------+------+--------+-----+----+----+------+--------+---+---+---+--------------------+
|    GP|  F| 18|      U|    GT3|      A|         4|   4|at_home| teacher|course|  mother|         2|        2|       0|      yes|    no|  no|        no|    yes|   yes|      no|      no|     4|       

##**RUNNING SQL QUERIES PROGRAMMATICALLY**

Registering Tables

In [45]:
# Registering a table
dataframe.registerTempTable("df")

/usr/local/lib/python3.7/dist-packages/pyspark/sql/dataframe.py:140: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  FutureWarning


In [46]:
sc.sql("select * from df").show(3)

+------+---+---+-------+-------+-------+----------+----+-------+-------+------+--------+----------+---------+--------+---------+------+----+----------+-------+------+--------+--------+------+--------+-----+----+----+------+--------+---+---+---+--------------------+
|school|sex|age|address|famsize|Pstatus|Mother_Edu|Fedu|   Mjob|   Fjob|reason|guardian|traveltime|studytime|failures|schoolsup|famsup|paid|activities|nursery|higher|internet|romantic|famrel|freetime|goout|Dalc|Walc|health|absences| G1| G2| G3|          new_column|
+------+---+---+-------+-------+-------+----------+----+-------+-------+------+--------+----------+---------+--------+---------+------+----+----------+-------+------+--------+--------+------+--------+-----+----+----+------+--------+---+---+---+--------------------+
|    GP|  F| 18|      U|    GT3|      A|         4|   4|at_home|teacher|course|  mother|         2|        2|       0|      yes|    no|  no|        no|    yes|   yes|      no|      no|     4|       3|  